# HW2 Problem9

In [1]:
import getpass
import os

if not os.environ.get("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key: ")


Enter your OpenAI API key: ··········


In [2]:
import os
from getpass import getpass

os.environ["OPENAI_API_KEY"] = getpass("Enter your OpenAI API key: ")


Enter your OpenAI API key: ··········


In [3]:
%pip install -qU langchain-openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.4/70.4 kB 3.4 MB/s eta 0:00:00


In [4]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model="gpt-4o",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # api_key="...",  # if you prefer to pass api key in directly instaed of using env vars
    # base_url="...",
    # organization="...",
    # other params...
)

In [16]:
from langchain_core.prompts import ChatPromptTemplate

system_message = "Classify the sentiment of the following text as Positive, Negative, or Neutral. Only reply with one of: Positive, Negative, or Neutral."
prompt_template = ChatPromptTemplate.from_messages([
    ("system", system_message),
    ("human", "{input}")
])


## 1A


In [17]:
texts = [
    "The movie was absolutely fantastic and exceeded my expectations!",
    "I'm not sure how I feel about this product.",
    "This service was terrible and completely disappointing."
]

# Prepare your LLM (replace with your model init code, e.g. gpt-4o)
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-4o")  # or "gpt-4", "gpt-4-turbo", etc.

# Run the model for each example
for text in texts:
    prompt = prompt_template.invoke({"input": text})
    result = llm.invoke(prompt.messages)
    print(f"Text: {text}\nSentiment: {result.content}\n")


Text: The movie was absolutely fantastic and exceeded my expectations!
Sentiment: Positive

Text: I'm not sure how I feel about this product.
Sentiment: Neutral

Text: This service was terrible and completely disappointing.
Sentiment: Negative



## 1B

In [18]:
system_fewshot = """Classify the sentiment of the following text as Positive, Negative, or Neutral. Only reply with one of: Positive, Negative, or Neutral.

Examples:
Text: "I loved this product, it works great!"
Sentiment: Positive

Text: "It was okay, nothing special."
Sentiment: Neutral

Text: "I hated the service, it was awful."
Sentiment: Negative
"""
prompt_template_few = ChatPromptTemplate.from_messages([
    ("system", system_fewshot),
    ("human", "{input}")
])


In [19]:
for text in texts:
    prompt = prompt_template_few.invoke({"input": text})
    result = llm.invoke(prompt.messages)
    print(f"Text: {text}\nSentiment: {result.content}\n")


Text: The movie was absolutely fantastic and exceeded my expectations!
Sentiment: Positive

Text: I'm not sure how I feel about this product.
Sentiment: Neutral

Text: This service was terrible and completely disappointing.
Sentiment: Negative



## 1C
From this result we are very hard to tell the difference. But usualy a few-shot prompt would do better. Few-shot prompting gives the model concrete examples of what you want—including the style, format, and boundaries for answers.
In zero-shot, the model has to guess your intent based only on a generic instruction.

In few-shot, you “anchor” the model’s behavior with labeled data points, making it less likely to misinterpret your prompt.

Few-shot examples can include tricky or ambiguous cases, so the model learns how to respond to them.

It also teaches the model exactly how you want the output formatted (e.g., “Sentiment: Positive” vs just “Positive”).

LLMs were trained on huge datasets with many different prompt styles.

By providing a few labeled examples, you help the model activate the correct pattern or template in its parameters, making the answer more reliable.



## 2A

In [21]:
# 1. CoT Prompt
cot_system = (
    "Let's solve math problems step by step. Show all calculations and reasoning."
)
cot_human = (
    "A bakery sells cupcakes for $3 each and cookies for $2 each. "
    "If Sarah buys 4 cupcakes and 6 cookies, and pays with a $50 bill, how much change will she receive?"
)

cot_prompt = ChatPromptTemplate.from_messages([
    ("system", cot_system),
    ("human", cot_human)
])

llm = ChatOpenAI(model="gpt-4o")  # Or your preferred model

# Run CoT (CORRECT WAY)
prompt = cot_prompt.invoke({})
cot_result = llm.invoke(prompt.messages)
print("Chain-of-Thought Result:\n", cot_result.content)


Chain-of-Thought Result:
 To find out how much change Sarah will receive, we first need to calculate the total cost of the cupcakes and cookies she buys.

1. **Calculate the cost of the cupcakes:**
   Sarah buys 4 cupcakes, and each cupcake costs $3.
   \[
   \text{Cost of cupcakes} = 4 \times 3 = 12 \text{ dollars}
   \]

2. **Calculate the cost of the cookies:**
   Sarah buys 6 cookies, and each cookie costs $2.
   \[
   \text{Cost of cookies} = 6 \times 2 = 12 \text{ dollars}
   \]

3. **Find the total cost:**
   Add the cost of cupcakes and the cost of cookies.
   \[
   \text{Total cost} = \text{Cost of cupcakes} + \text{Cost of cookies} = 12 + 12 = 24 \text{ dollars}
   \]

4. **Calculate the change:**
   Sarah pays with a $50 bill. Subtract the total cost from the amount she pays to find the change.
   \[
   \text{Change} = 50 - 24 = 26 \text{ dollars}
   \]

Therefore, Sarah will receive \(\boxed{26}\) dollars in change.


## 2B


In [23]:
direct_system = (
    "Answer the following math question as concisely as possible."
)
direct_human = cot_human  # same question

direct_prompt = ChatPromptTemplate.from_messages([
    ("system", direct_system),
    ("human", direct_human)
])

# Run Direct
direct_prompt = direct_prompt.invoke({})
direct_result = llm.invoke(direct_prompt.messages)
print("Direct Result:\n", direct_result.content)


Direct Result:
 Sarah buys 4 cupcakes at $3 each, totaling \( 4 \times 3 = 12 \) dollars. She also buys 6 cookies at $2 each, totaling \( 6 \times 2 = 12 \) dollars. The total cost of her purchase is \( 12 + 12 = 24 \) dollars. Paying with a $50 bill, she receives \( 50 - 24 = 26 \) dollars in change. Thus, Sarah will receive $26 in change.


## 2C
I think the CoT result is more reliable. the result is "Sarah will receive $26 in change."

Why is CoT usually more reliable?
1. Forces Step-by-Step Reasoning
CoT prompts explicitly tell the model to “think step by step.”

This helps prevent the model from jumping straight to an answer (which is where it most often makes mistakes).

2. Reduces "Shortcut" Errors
LLMs sometimes use pattern-matching shortcuts to guess an answer if asked directly.

When you ask for steps, the model must “show its work,” making it less likely to overlook details.

3. Reveals Model's Reasoning
You can see how the model got its answer.

If there’s a mistake, it’s easier to spot (and fix) because you see all the steps.

4. Activates Training Patterns
During training, LLMs saw a lot of “let’s think step by step” and similar reasoning patterns.

CoT prompts match those patterns, so the model “knows” to be more careful and explicit.

5. Works Especially Well for Multi-Step Tasks
The more steps required, the more CoT helps.

For math word problems, logical deduction, and coding, CoT prompting is much more robust.



## 3A

In [24]:
prompt_text = (
    "Let's solve the following logic problem step by step:\n\n"
    "In a race with 5 participants (Alice, Bob, Carol, David, and Eve), "
    "Alice finished before Bob but after Carol. David finished last, and "
    "Eve finished before Alice but after Bob. What is the final ranking?"
)

prompt = ChatPromptTemplate.from_messages([
    ("system", "Solve logic puzzles step by step. Show your reasoning and then give the final ranking as a list from 1st to 5th."),
    ("human", prompt_text)
])

temperatures = [0.3, 0.7, 1.0]
results = []

for temp in temperatures:
    llm = ChatOpenAI(model="gpt-4o", temperature=temp)
    prompt_instance = prompt.invoke({})
    response = llm.invoke(prompt_instance.messages)
    print(f"\nTemperature: {temp}\n{response.content}\n")
    results.append((temp, response.content))



Temperature: 0.3
To solve the puzzle, let's analyze the given information step by step:

1. **Alice finished before Bob but after Carol.**
   - This means Carol finished before Alice, and Alice finished before Bob.
   - So, the order among these three is: Carol > Alice > Bob.

2. **David finished last.**
   - David is in the 5th position.

3. **Eve finished before Alice but after Bob.**
   - This means Bob finished before Eve, and Eve finished before Alice.
   - So, the order among these three is: Bob > Eve > Alice.

Now, let's combine all these pieces of information:

- From the first point, we have: Carol > Alice > Bob.
- From the third point, we have: Bob > Eve > Alice.

Combining these, we need to find a sequence that satisfies all conditions:

- Since Carol is before Alice and Bob is before Eve, and Eve is before Alice, the only possible order is:
  - Carol > Bob > Eve > Alice > David.

Thus, the final ranking is:
1. Carol
2. Bob
3. Eve
4. Alice
5. David


Temperature: 0.7
To sol

## 3C
All three temperature settings led to the same final ranking.

The model's step-by-step reasoning correctly integrated all the logical constraints, even as temperature increased.

Self-consistency is high in this case: the answer did not change with temperature, showing robustness.



## 4


## A

In [26]:
explore_prompt = ChatPromptTemplate.from_messages([
    ("system",
     "You are a creative camping expert. For the given constraints, brainstorm several different strategies for selecting the 5 most important items. For each, briefly explain your reasoning and then list your 5 chosen items."),
    ("human",
     "You need to pack for a 3-day camping trip. You have a backpack that can hold 5 items maximum. "
     "Available items: tent, sleeping bag, food (3 days worth), water bottles, flashlight, first aid kit, map, extra clothes, cooking equipment. "
     "List a few different possible packing strategies. For each, explain your reasoning and list your 5 chosen items.")
])

llm = ChatOpenAI(model="gpt-4o", temperature=0.7)
prompt_instance = explore_prompt.invoke({})
response = llm.invoke(prompt_instance.messages)
print(response.content)

Certainly! Here are a few different strategies for selecting the 5 most essential items for a 3-day camping trip:

### Strategy 1: Survival Essentials
**Reasoning:** Prioritize the most critical items for basic survival, focusing on shelter, warmth, hydration, and food.

1. **Tent** - Provides essential shelter and protection from the elements.
2. **Sleeping Bag** - Ensures warmth and comfort during the night.
3. **Water Bottles** - Crucial for staying hydrated throughout the trip.
4. **Food (3 days worth)** - Necessary to maintain energy and health.
5. **First Aid Kit** - Essential for handling any minor injuries or health issues.

### Strategy 2: Lightweight and Efficient
**Reasoning:** Select items that are lightweight and multifunctional, maximizing efficiency and mobility.

1. **Tent** - Offers shelter while being relatively compact.
2. **Sleeping Bag** - Lightweight and compact for easy carrying.
3. **Water Bottles** - Stay hydrated without the bulk.
4. **Food (3 days worth)** - 

## B

In [25]:
tot_prompt = ChatPromptTemplate.from_messages([
    ("system",
     "You are an expert in planning camping trips. Given the constraints, list several different possible strategies to choose the best items, each based on a different priority. For each, explain your reasoning, then list the 5 chosen items."),
    ("human",
     "You need to pack for a 3-day camping trip. You have a backpack that can hold 5 items maximum. Available items: tent, sleeping bag, food (3 days worth), water bottles, flashlight, first aid kit, map, extra clothes, cooking equipment.\n\nGenerate three possible packing plans:\n1. Safety-first\n2. Comfort-first\n3. Minimal-weight\n\nFor each plan, briefly explain your reasoning and list your 5 chosen items.")
])
llm = ChatOpenAI(model="gpt-4o", temperature=0.7)

prompt_instance = tot_prompt.invoke({})
response = llm.invoke(prompt_instance.messages)
print(response.content)

### Safety-first Plan:
**Reasoning:** The priority here is to ensure that you are prepared for any emergencies or unexpected situations that might arise during the trip. Items are selected based on their ability to provide shelter, help navigate, and deal with potential injuries.

1. **Tent** - Provides shelter against the elements, which is crucial for safety.
2. **First Aid Kit** - Essential for treating injuries and ensuring health during the trip.
3. **Map** - Critical for navigation and ensuring you don't get lost.
4. **Food (3 days worth)** - Ensures sustenance and energy for the duration of the trip.
5. **Water Bottles** - Hydration is vital for survival and maintaining health.

### Comfort-first Plan:
**Reasoning:** This plan focuses on maximizing comfort and convenience during the camping trip. The choices prioritize a pleasurable and relaxing experience.

1. **Tent** - Offers comfort and protection while sleeping outdoors.
2. **Sleeping Bag** - Ensures a warm and comfortable 

## C analysis:
Part 1: Exploring Multiple Approaches (Unlabeled Branches)
The model brainstormed several strategies:

Survival Essentials: Focused on shelter, warmth, hydration, food, first aid.

Lightweight and Efficient: Chose items that are light and multifunctional, favoring a flashlight over first aid.

Comfort and Convenience: Prioritized extra clothes and cooking equipment.

Emergency Preparedness: Emphasized navigation (map) and first aid.

Balanced Approach: A compromise including flashlight for convenience.

Observation:
Each approach emerged naturally from the LLM’s “thinking,” and represents different real-world priorities—even though priorities were not stated explicitly. There is overlap in core items (tent, sleeping bag, food, water), but the fifth item changes based on the approach.

Part 2: Explicit Branching by Priority
Safety-first:

Reasoning: Prioritized emergencies, navigation, and survival.

Items: Tent, first aid kit, map, food, water bottles.

Comfort-first:

Reasoning: Maximized sleeping comfort, meal enjoyment, and clothing changes.

Items: Tent, sleeping bag, food, cooking equipment, extra clothes.

Minimal-weight:

Reasoning: Chose lightest essentials, avoided heavy or redundant items.

Items: Sleeping bag, food, water bottles, flashlight, first aid kit.

Observation:
Explicitly telling the model to “branch” by priority made the differences sharper and reasoning more systematic. For example, map appears in safety-first but not in comfort/minimal, and cooking equipment is only in comfort-first.

Comparison & Evaluation
Unlabeled branches capture diverse real-world strategies, showing the model’s creativity in problem-solving.

Priority-labeled branches are clearer for decision-making and easier to compare, since reasoning for each priority is explicitly justified.

Both approaches consistently included essentials (shelter, food, water), but differed in their 4th or 5th choice depending on the “thought branch.”

Which approach is best?

For real-world planning, priority-labeled ToT makes it easier to choose a plan based on your specific needs (safety, comfort, weight).

The open-ended (unlabeled) ToT is better for discovering unexpected or creative solutions.



## 5A

In [29]:
retrieval_content = """
Solar:
- Advantages: Abundant and inexhaustible resource, minimal operating costs after installation, relatively low maintenance, and can be installed in various locations.
- Disadvantages: Intermittent energy production (dependent on sunlight), high initial installation costs, can be aesthetically unappealing, and requires significant land area.

Wind:
- Advantages: Clean and renewable source, low operating costs, can be scaled for different needs, and can be located offshore.
- Disadvantages: Intermittent energy generation (dependent on wind), potential visual and noise pollution, and can impact wildlife.

Hydro:
- Advantages: Reliable and steady if freshwater is available, can provide energy storage (via pumped storage), long lifespan. Most commercially available renewable source, reliable and flexible, can be used to power rural areas, and can be relatively inexpensive in the long run.
- Disadvantages: High upfront construction costs (dams, etc.), potential environmental impacts on aquatic habitats and local communities, and limited locations due to geographical requirements (e.g., consistent water flow).
"""

## 5B

In [30]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

rag_prompt = ChatPromptTemplate.from_messages([
    ("system",
     "You are an expert energy consultant. Use only the information in the retrieved document below to answer the user's question."),
    ("human",
     f"Retrieved document:\n{retrieval_content}\n\nQuestion: What are the advantages and disadvantages of different renewable energy sources, and which would be most suitable for a small island nation?")
])

llm = ChatOpenAI(model="gpt-4o", temperature=0)
prompt_instance = rag_prompt.invoke({})
response_rag = llm.invoke(prompt_instance.messages)
print(response_rag.content)


When considering renewable energy sources for a small island nation, it's important to weigh the advantages and disadvantages of each option:

**Solar Energy:**
- *Advantages:* Solar energy is abundant and inexhaustible, with minimal operating costs after installation. It requires relatively low maintenance and can be installed in various locations, making it versatile for different settings.
- *Disadvantages:* Solar energy production is intermittent, as it depends on sunlight, and the initial installation costs can be high. It may also require significant land area and can be aesthetically unappealing.

**Wind Energy:**
- *Advantages:* Wind energy is a clean and renewable source with low operating costs. It can be scaled to meet different energy needs and can be located offshore, which might be beneficial for an island nation.
- *Disadvantages:* Like solar, wind energy generation is intermittent, depending on wind availability. It can also cause visual and noise pollution and may impa

## 5C

In [31]:
no_rag_prompt = ChatPromptTemplate.from_messages([
    ("system",
     "You are an expert energy consultant. Answer the user's question using your own knowledge."),
    ("human",
     "What are the advantages and disadvantages of different renewable energy sources (solar, wind, hydro), and which would be most suitable for a small island nation?")
])

prompt_instance = no_rag_prompt.invoke({})
response_norag = llm.invoke(prompt_instance.messages)
print(response_norag.content)


When considering renewable energy sources for a small island nation, it's important to evaluate the advantages and disadvantages of each option and consider the specific characteristics and needs of the island. Here's a breakdown of solar, wind, and hydro energy:

### Solar Energy

**Advantages:**
- **Abundant Resource:** Solar energy is plentiful in many regions, especially in areas with high sun exposure.
- **Scalability:** Solar panels can be installed on a small scale (e.g., rooftops) or large scale (e.g., solar farms).
- **Low Operating Costs:** Once installed, solar panels require minimal maintenance and have no fuel costs.
- **Modular Installation:** Solar systems can be expanded easily as demand grows.

**Disadvantages:**
- **Intermittency:** Solar power is dependent on sunlight, which can be inconsistent due to weather and day-night cycles.
- **Initial Costs:** The upfront cost of solar panels and installation can be high, although prices have been decreasing.
- **Space Requir

## 5C
The main difference is that, first, RAG simulations gives an answer like "we can decide which one to choice according to the features of the island." However, the pretrain model gives answer like " we can have a combination of all of them." which might not be the answer we really want. Secondly, the RAG simualtion gives answer in a short way. It uses the text we provided to answer. While the pretrained model gives a very long answer. It trys to gather the information first.

## 5D
Benefits of Using External Knowledge (RAG) in Prompt Engineering
Accuracy & Trust: RAG allows you to control the information the model can access, ensuring the answer is grounded in facts you provide.

Freshness: Retrieval enables up-to-date knowledge, even if the LLM is out-of-date.

Domain Adaptation: Easily add custom, domain-specific facts that may not be present in pre-trained weights.

Transparency: You can audit the “retrieved” content, making it easy to debug and trust the reasoning.

Safer Outputs: Reduces risk of hallucination, as the model is limited to retrieved facts.



## 6A

In [6]:
!pip install langchain langchain-openai google-search-results

  Preparing metadata (setup.py) ... done
  Created wheel for google-search-results: filename=google_search_results-2.4.2-py3-none-any.whl size=32010 sha256=d691bb1639d71e647164f7bf6c764b32557f84bc752c192618884751ac3ca387
  Stored in directory: /root/.cache/pip/wheels/6e/42/3e/aeb691b02cb7175ec70e2da04b5658d4739d2b41e5f73cd06f
Successfully built google-search-results


In [8]:
!pip install langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 55.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 6.1 MB/s eta 0:00:00


In [12]:
from langchain_openai import ChatOpenAI
from langchain.agents import initialize_agent, Tool
from langchain.agents.agent_types import AgentType




In [16]:
from langchain_community.utilities import SerpAPIWrapper


In [19]:
import os
os.environ["SERPAPI_API_KEY"] = "252adbec8485f8f8f5c74c7b847518c4caefdcf8d3fe0ac5906f77a62fe4f985"


In [20]:
from langchain_community.agent_toolkits.load_tools import load_tools
tools = load_tools(["serpapi"])

In [22]:
# 3. Initialize tools
search = SerpAPIWrapper()   # Web search tool
tools = load_tools(['serpapi', 'llm-math'], llm=ChatOpenAI())  # Includes search and calculator

In [23]:
# 4. Initialize LLM
llm = ChatOpenAI(model="gpt-3.5-turbo")  # or "gpt-4o" if you have access

# 5. Initialize agent with tools
agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

# 6. Multi-step question for the agent
question = (
    "What is the population of the largest city in Japan, "
    "and what would be 15% of that population?"
)

# 7. Run the agent and print result
response = agent.run(question)
print(response)

/tmp/ipython-input-23-640519425.py:5: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(
/tmp/ipython-input-23-640519425.py:19: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = agent.run(question)




> Entering new AgentExecutor chain...
I can use a search engine to find the population of the largest city in Japan and then a calculator to calculate 15% of that population. 
Action: Search
Action Input: Population of largest city in Japan
Observation: ['Tokyo, with almost nine million inhabitants, is by far the largest Japanese city. (The Greater Tokyo region is estimated to have a population of some 13 million) ...', 'The largest city in Japan is Tokyo, with a population of 8,336,599 people. Despite being just the 61st largest country in the world by area, Japan is ranked ...', '1, Tokyo · Tokyo, 9,272,740, 11 ...', "With approximately 9.57 million inhabitants, Tokyo was Japan's most populous city as of 2023, followed by Yokohama, which, in the same year, ...", 'Tokyo, officially the Tokyo Metropolis, is the capital and most populous city in Japan. With a population of over 14 million in the city proper in 2023, ...', 'Tokyo, Japan, is the largest city on Earth, with a population 

## 6B

In [24]:

# Initialize the LLM (standalone, no tools)
llm = ChatOpenAI(model="gpt-3.5-turbo")  # Or "gpt-4o" if you have access

# Standalone prompt (no tools or retrieval)
question = (
    "What is the population of the largest city in Japan, and what would be 15% of that population? "
    "Please show your reasoning."
)

# Ask the LLM directly
response = llm.invoke(question)
print(response.content)

The largest city in Japan is Tokyo, with a population of approximately 9.73 million as of 2021.

To find 15% of the population of Tokyo, we can use the formula:

15% of 9.73 million = 0.15 x 9,730,000 = 1,459,500

Therefore, 15% of the population of Tokyo would be approximately 1,459,500 people.


Tool-Augmented LLMs (e.g., ReAct agent with SerpAPI + Calculator)
Advantages:

Access to Real-Time and External Data:
The LLM can fetch up-to-date information from the web, databases, or perform live calculations.
Example: Your agent fetched the actual current population of Tokyo using a search engine, then calculated 15% programmatically.

Increased Accuracy:
Answers are grounded in real data, reducing errors due to outdated or memorized knowledge.

Enhanced Capabilities:
Can solve multi-step or tool-based tasks that go beyond the LLM’s own “memory” (e.g., currency conversions, mathematical calculations, code execution).

Transparency in Reasoning:
The reasoning chain and tool usage steps are visible, allowing for easier debugging and validation.

Limitations:

Dependency on Tool Availability:
Requires integration and access to APIs or external tools, which might need keys, credits, or may go offline.

Latency:
Querying external tools can slow down response time compared to instant LLM answers.

Complexity:
Setting up agents, tools, and API credentials adds engineering overhead.

Standalone Prompting (LLM Only, No Tools)
Advantages:

Simplicity:
No extra setup or integration—just a single prompt to the model.

Speed:
The model generates a response instantly, without waiting for external API calls.

Low Overhead:
No need to manage API keys, credits, or tool-specific code.

Limitations:

Stale or Inaccurate Knowledge:
The LLM relies on its training data, which may be outdated (e.g., population numbers from 2021 or earlier).

Estimates, Not Facts:
Answers may be “best guesses” based on its training, not live data. This can lead to errors, especially in fast-changing domains (e.g., news, prices, population).

No Programmatic Calculation:
For arithmetic or logic, the model simulates reasoning but can make mistakes or hallucinate.



Tool-Augmented:
Gave the latest population (8,336,599 for Tokyo), used a calculator tool, and output: 1,250,489 (15% of that).

Standalone:
Used its memorized population (9.73 million, as of 2021), calculated 15% as 1,459,500.